<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Sprint Challenge
## *Data Science Unit 4 Sprint 1*

After a week of Natural Language Processing, you've learned some cool new stuff: how to process text, how turn text into vectors, and how to model topics from documents. Apply your newly acquired skills to one of the most famous NLP datasets out there: [Yelp](https://www.yelp.com/dataset/challenge). As part of the job selection process, some of my friends have been asked to create analysis of this dataset, so I want to empower you to have a head start.  

The real dataset is massive (almost 8 gigs uncompressed). I've sampled the data for you to something more managable for the Sprint Challenge. You can analyze the full dataset as a stretch goal or after the sprint challenge. As you work on the challenge, I suggest adding notes about your findings and things you want to analyze in the future.

## Challenge Objectives
*Successfully complete these all these objectives to earn a 2. There are more details on each objective further down in the notebook.*
* <a href="#p1">Part 1</a>: Write a function to tokenize the yelp reviews
* <a href="#p2">Part 2</a>: Create a vector representation of those tokens
* <a href="#p3">Part 3</a>: Use your tokens in a classification model on yelp rating
* <a href="#p4">Part 4</a>: Estimate & Interpret a topic model of the Yelp reviews

In [1]:
import pandas as pd

yelp = pd.read_json('./data/review_sample.json', lines=True)

In [2]:
yelp.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,nDuEqIyRc8YKS1q1fX0CZg,1,2015-03-31 16:50:30,0,eZs2tpEJtXPwawvHnHZIgQ,1,"BEWARE!!! FAKE, FAKE, FAKE....We also own a sm...",10,n1LM36qNg4rqGXIcvVXv8w
1,eMYeEapscbKNqUDCx705hg,0,2015-12-16 05:31:03,0,DoQDWJsNbU0KL1O29l_Xug,4,Came here for lunch Togo. Service was quick. S...,0,5CgjjDAic2-FAvCtiHpytA
2,6Q7-wkCPc1KF75jZLOTcMw,1,2010-06-20 19:14:48,1,DDOdGU7zh56yQHmUnL1idQ,3,I've been to Vegas dozens of times and had nev...,2,BdV-cf3LScmb8kZ7iiBcMA
3,k3zrItO4l9hwfLRwHBDc9w,3,2010-07-13 00:33:45,4,LfTMUWnfGFMOfOIyJcwLVA,1,We went here on a night where they closed off ...,5,cZZnBqh4gAEy4CdNvJailQ
4,6hpfRwGlOzbNv7k5eP9rsQ,1,2018-06-30 02:30:01,0,zJSUdI7bJ8PNJAg4lnl_Gg,4,"3.5 to 4 stars\n\nNot bad for the price, $12.9...",5,n9QO4ClYAS7h9fpQwa5bhA


## Part 1: Tokenize Function
<a id="#p1"></a>

Complete the function `tokenize`. Your function should
- accept one document at a time
- return a list of tokens

You are free to use any method you have learned this week.

In [15]:
import re
import spacy

nlp = spacy.load("en_core_web_lg")
spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

def tokenize(x):
    text = x.lower()
    text = re.sub(r'[^a-zA-Z ^0-9]', '', str(text))
    return text.split()

def spacy_lemmatize(x):
    doc = nlp.tokenizer(x)
    return [token.lemma_ for token in doc]


def remove_stopwords(tokens):
    cleaned_tokens = []
    for token in tokens:
        if token not in spacy_stopwords:
            cleaned_tokens.append(token)
    return ' '.join(cleaned_tokens)


yelp['tokens'] = yelp['text'].apply(lambda x: tokenize(x))
yelp['tokens'] = yelp['tokens'].apply(lambda x: spacy_lemmatize(' '.join(x)))
yelp['tokens'] = yelp['tokens'].apply(lambda x: remove_stopwords(x))

In [16]:
yelp['tokens'].head()

0    beware fake fake fakewe small business los ala...
1    come lunch togo service quick staff friendly c...
2    -PRON- vega dozen time step foot circus circus...
3    night close street party actually group guy pa...
4    35 4 starsnot bad price 1299 lunch senior 15 p...
Name: tokens, dtype: object

In [67]:
yelp.shape

(10000, 10)

## Part 2: Vector Representation
<a id="#p2"></a>
1. Create a vector representation of the reviews
2. Write a fake review and query for the 10 most similiar reviews, print the text of the reviews. Do you notice any patterns?
    - Given the size of the dataset, it will probably be best to use a `NearestNeighbors` model for this. 

In [97]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_features = 5000)
dtm = tfidf.fit_transform(yelp['tokens'])
# dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())
docs = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
docs.head()

,10,100,1000,101,1010,1015,1030,11,110,1100,...,yuk,yum,yummy,yup,zen,zero,zombie,zone,zoo,zucchini
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [98]:
docs.index = yelp.index

In [99]:
# Instantiate
from sklearn.neighbors import NearestNeighbors

# Fit on TF-IDF Vectors
nn  = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(docs)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [100]:
# Query Using kneighbors 
nn.kneighbors([docs.iloc[5]])

(array([[0.        , 1.        , 1.        , 1.1270721 , 1.19984008,
         1.22720223, 1.22827212, 1.24064974, 1.24591349, 1.25798238]]),
 array([[   5, 6204, 6311, 9884, 9173, 4126, 8720, 2561, 7949, 6714]],
       dtype=int64))

In [101]:
yelp['tokens'][4126][:300]

'solid place live nearby good place drop feel like cook -PRON- try flavourful arepa reina chickenavocado arepa tasty mayo avocado ratio wayy mayo avocado beef empanada good size tasty portion decent price'

In [113]:
fake_review = ["this is a great restaurant, best tofu I've ever had"]

In [114]:
check = tfidf.transform(fake_review)

nn.kneighbors(check.todense())

(array([[1.        , 1.        , 1.00520665, 1.04659866, 1.07097516,
         1.0779505 , 1.10429261, 1.14054301, 1.15377508, 1.16447427]]),
 array([[6204, 6311, 3922, 6340, 3728, 5332, 7735, 1893, 6028, 5374]],
       dtype=int64))

In [115]:
x = [6311, 6204, 8697, 9136, 1734, 9241, 1542, 1550, 3903, 7593]

for i in x:
    print(yelp['tokens'][i][:500] + "\n")
    
# Trash was apparently an operative word. The first two blank reviews came through, and that's problematic.
# I'd need to clean that up going forward, but this works well enough to be MVP...





fast good price drink good light staff kind helpful outside smell like trash trash right -PRON- come area hungry hibachi sushi record

bomb trash nachos killer bloody mary great service locate right heart strip favorite mtley q sandwich different time trash nachos soooo good large portion share 4 definitely

dan fix chair think trash tell rear

nice trash place employee look like want drop food manager look look away customer service whatsoever hey -PRON- las vega care right

trash 3 people work trash leg paper towel like night swing shift push towel rhe spray bottle lol o good hopefully pf improve om gym clean good bathroom

boyfriend eat local meat product hard find area catch open excite product shrimp twice excellent time love local product like pasta bread etc nice able walk run good work ill

love fresh easy shop 3 time week great customer service quality product check help unlike lot big chain grocery store hard find busy appreciate fact quickly open 24 hour start carry wild

## Part 3: Classification
<a id="#p3"></a>
Your goal in this section will be to predict `stars` from the review dataset. 

1. Create a piepline object with a sklearn `CountVectorizer` or `TfidfVector` and any sklearn classifier. Use that pipeline to estimate a model to predict `stars`. Use the Pipeline to predict a star rating for your fake review from Part 2. 
2. Tune the entire pipeline with a GridSearch

In [107]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

In [108]:
vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [109]:
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])

In [110]:
parameters = {
    'vect__max_df': ( 0.9, 1.0), #max_df stands for drop words that have to appear over a certain threshold to be excluded
    'vect__min_df': (.02, .05), #min_df is the minimum percentage of documents a feature must appear in to be included
    'vect__max_features': (1000,5000),
    'clf__n_estimators':(5, 10,),
    'clf__max_depth':(15,20)
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(yelp['tokens'], yelp['stars'])

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  1.7min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.9, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (1000, 5000), 'clf__n_estimators': (5, 10), 'clf__max_depth': (15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [111]:
grid_search.best_score_
# doesn't seem like a great score, but after playing with some of the parameters this didn't move much...

0.5362

In [112]:
grid_search.predict(["this is a great restaurant, best tofu I've ever had"])

array([5], dtype=int64)

## Part 4: Topic Modeling

Let's find out what those yelp reviews are saying! :D

1. Estimate a LDA topic model of the review text
    - Keep the `iterations` parameter at or below 5 to reduce run time
    - The `workers` parameter should match the number of physical cores on your machine.
2. Create 1-2 visualizations of the results
    - You can use the most important 3 words of a topic in relevant visualizations. Refer to yesterday's notebook to extract. 
3. In markdown, write 1-2 paragraphs of analysis on the results of your topic model

__*Note*__: You can pass the DataFrame column of text reviews to gensim. You do not have to use a generator.

In [68]:
from gensim.models import LdaMulticore
from gensim.corpora import Dictionary

C:\Users\ltown\Anaconda3\envs\U4-S1-NLP\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Learn the vocubalary of the yelp data:

In [76]:
def split(x):
    return x.split()

yelp['vocab'] = yelp['tokens'].apply(lambda x: split(x))
# I needed to break up the docs into invidual words

In [77]:
id2word = Dictionary(yelp['vocab'])

Create a bag of words representation of the entire corpus

In [78]:
corpus = [id2word.doc2bow(text) for text in yelp['vocab']]

Your LDA model should be ready for estimation: 

In [79]:
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   iterations=5,
                   workers=4,
                   num_topics = 10 # You can change this parameter
                  )

Create 1-2 visualizations of the results

In [80]:
import pyLDAvis.gensim

pyLDAvis.enable_notebook()

In [81]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

C:\Users\ltown\Anaconda3\envs\U4-S1-NLP\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.004607 -0.001268       1        1  10.712153
6      0.001698 -0.002214       2        1  10.384345
0      0.000283  0.000559       3        1  10.173436
9      0.002124 -0.001051       4        1  10.143820
1     -0.002750 -0.003974       5        1   9.982859
4     -0.005116  0.000781       6        1   9.930052
2     -0.002245  0.000083       7        1   9.927717
5      0.002018  0.001201       8        1   9.715178
3     -0.001061  0.000065       9        1   9.621071
8      0.000443  0.005817      10        1   9.409374, topic_info=     Category         Freq        Term        Total  loglift  logprob
42    Default  5985.000000        good  5985.000000  30.0000  30.0000
204   Default  4041.000000       great  4041.000000  29.0000  29.0000
129   Default  4965.000000       place  4965.000000  28.0000  28.0000
36    Default  4011.000000        time  4011.000000  27.0000  27.0000
47    Default  3459.000000     service  3459.000000  26.0000  26.0000
52    Default  5093.000000      -PRON-  5093.000000  25.0000  25.0000
39    Default  3689.000000        come  3689.000000  24.0000  24.0000
20    Default  3558.000000        like  3558.000000  23.0000  23.0000
665   Default  2113.000000        love  2113.000000  22.0000  22.0000
482   Default  1445.000000        tell  1445.000000  21.0000  21.0000
1144  Default  1294.000000     chicken  1294.000000  20.0000  20.0000
348   Default  1515.000000        find  1515.000000  19.0000  19.0000
404   Default  1433.000000         ask  1433.000000  18.0000  18.0000
364   Default  3211.000000       order  3211.000000  17.0000  17.0000
299   Default  1718.000000        want  1718.000000  16.0000  16.0000
41    Default  1298.000000    friendly  1298.000000  15.0000  15.0000
144   Default  4609.000000        food  4609.000000  14.0000  14.0000
57    Default  1000.000000         bar  1000.000000  13.0000  13.0000
147   Default  1063.000000         fry  1063.000000  12.0000  12.0000
190   Default  1290.000000       amaze  1290.000000  11.0000  11.0000
27    Default   992.000000      minute   992.000000  10.0000  10.0000
243   Default  1015.000000       right  1015.000000   9.0000   9.0000
127   Default  1334.000000      people  1334.000000   8.0000   8.0000
266   Default  1165.000000        feel  1165.000000   7.0000   7.0000
225   Default   553.000000       thank   553.000000   6.0000   6.0000
315   Default  1428.000000        work  1428.000000   5.0000   5.0000
879   Default   546.000000    sandwich   546.000000   4.0000   4.0000
154   Default  1706.000000       price  1706.000000   3.0000   3.0000
92    Default  1689.000000        nice  1689.000000   2.0000   2.0000
681   Default  1155.000000       taste  1155.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
20    Topic10   368.560181        like  3558.273926   0.0960  -4.8071
129   Topic10   487.554779       place  4965.479980   0.0426  -4.5273
976   Topic10   124.766373         use  1047.395020   0.2358  -5.8903
839   Topic10   108.655769       clean   906.219971   0.2424  -6.0286
436   Topic10    76.287323        help   605.378174   0.2921  -6.3822
309   Topic10    53.361504         ill   400.839294   0.3470  -6.7397
101   Topic10   172.276382  restaurant  1595.010498   0.1379  -5.5676
144   Topic10   409.292999        food  4609.797363  -0.0580  -4.7023
92    Topic10   177.180099        nice  1689.028320   0.1087  -5.5396
10    Topic10    63.376816    business   495.244568   0.3075  -6.5676
52    Topic10   434.556854      -PRON-  5093.457520  -0.0979  -4.6424
47    Topic10   310.510101     service  3459.519531  -0.0472  -4.9785
952   Topic10   124.447716         new  1129.778931   0.1576  -5.8929
153   Topic10   112.689079      pretty  1001.277100   0.1791  -5.9921
364   Topic10   284.647675       order  3211.693604  -0.0598  -5.0655
64    Topic10    95

## Stretch Goals

Complete one of more of these to push your score towards a three: 
* Incorporate named entity recognition into your analysis
* Compare vectorization methods in the classification section
* Analyze more (or all) of the yelp dataset - this one is v. hard. 
* Use a generator object on the reviews file - this would help you with the analyzing the whole dataset.
* Incorporate any of the other yelp dataset entities in your analysis (business, users, etc.)